In [1]:
#checking tensorflow version

import tensorflow as tf
print(tf.__version__)

2.9.1


In [2]:
#importing libraries 

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers

In [3]:
# Cross Validation dataset links

#CV-1
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 1/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 1/Validation/"

#CV-2
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 2/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 2/Validation/"

#CV-3
train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 3/Training/"
val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 3/Validation/"

#CV-4
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 4/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 4/Validation/"

#CV-5
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 5/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 5/Validation/"

In [4]:
from tensorflow.keras import layers

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128, 
                                                                 )
val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128,
                                                                )

Found 31206 files belonging to 5 classes.


2023-01-06 13:42:43.300745: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 13:42:43.324508: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 13:42:43.324728: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 13:42:43.338083: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 7787 files belonging to 5 classes.


In [5]:
#Normalizing the data

nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

val_data = val_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

In [6]:
#import tensorflow.python.keras.applications.VGG16
base_model= tf.keras.applications.VGG16(weights= 'imagenet', input_shape=(256,256, 3), include_top=False) 

In [7]:
# Freezing the base_model for transfer learning
base_model.trainable= False

In [8]:
# Creating a new_model on the top
inputs= tf.keras.Input(shape= (256, 256, 3), name="input_layer")

In [9]:
x= base_model(inputs,training= False)

In [10]:
# Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name='Global_Average_Pooling_Layer')(x)

In [11]:
# Output layer
u = tf.keras.layers.Dense(1050, activation="relu")(x)
u = tf.keras.layers.Dense(512, activation="relu")(u)
u = tf.keras.layers.Dense(256, activation="relu")(u)
u = tf.keras.layers.Dense(128, activation="relu")(u)
outputs = tf.keras.layers.Dense(5, activation="softmax",name="output_layer")(u)

In [12]:
# Combine the model
model = tf.keras.Model(inputs, outputs) 

In [13]:
#printing model summary

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 Global_Average_Pooling_Laye  (None, 512)              0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 1050)              538650    
                                                                 
 dense_1 (Dense)             (None, 512)               538112    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                             

In [15]:
# Compile the model

from tensorflow.keras.metrics import SensitivityAtSpecificity,SpecificityAtSensitivity,Recall,Precision

model.compile(loss= tf.losses.categorical_crossentropy, 
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy',SensitivityAtSpecificity(0.5),SpecificityAtSensitivity(0.5),Recall(0.5),Precision(0.5)])

In [16]:
#defining callback 
class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99): 
          print("\nReached 99% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [17]:
# Fit the model
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("ECG_Spectrogram_Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=50, validation_data=val_data, validation_steps=len(val_data),callbacks = [callbacks,checkpoint])

Epoch 1/50


2023-01-06 13:43:03.408189: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-01-06 13:43:03.651605: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


244/244 [==============================] - ETA: 0s - loss: 1.1501 - accuracy: 0.5194 - sensitivity_at_specificity_1: 0.9066 - specificity_at_sensitivity_1: 0.8997 - recall_1: 0.3160 - precision_1: 0.7159
Epoch 1: val_accuracy improved from -inf to 0.63837, saving model to ECG_Spectrogram_Model.h5
244/244 [==============================] - 204s 789ms/step - loss: 1.1501 - accuracy: 0.5194 - sensitivity_at_specificity_1: 0.9066 - specificity_at_sensitivity_1: 0.8997 - recall_1: 0.3160 - precision_1: 0.7159 - val_loss: 0.9258 - val_accuracy: 0.6384 - val_sensitivity_at_specificity_1: 0.9649 - val_specificity_at_sensitivity_1: 0.9555 - val_recall_1: 0.4710 - val_precision_1: 0.7560
Epoch 2/50
244/244 [==============================] - ETA: 0s - loss: 0.8609 - accuracy: 0.6538 - sensitivity_at_specificity_1: 0.9780 - specificity_at_sensitivity_1: 0.9630 - recall_1: 0.5389 - precision_1: 0.7488
Epoch 2: val_accuracy improved from 0.63837 to 0.66996, saving model to ECG_Spectrogram_Model.h5
2

244/244 [==============================] - ETA: 0s - loss: 0.5640 - accuracy: 0.7893 - sensitivity_at_specificity_1: 0.9934 - specificity_at_sensitivity_1: 0.9923 - recall_1: 0.7452 - precision_1: 0.8312
Epoch 13: val_accuracy did not improve from 0.71478
244/244 [==============================] - 176s 722ms/step - loss: 0.5640 - accuracy: 0.7893 - sensitivity_at_specificity_1: 0.9934 - specificity_at_sensitivity_1: 0.9923 - recall_1: 0.7452 - precision_1: 0.8312 - val_loss: 0.7902 - val_accuracy: 0.7072 - val_sensitivity_at_specificity_1: 0.9792 - val_specificity_at_sensitivity_1: 0.9781 - val_recall_1: 0.6620 - val_precision_1: 0.7475
Epoch 14/50
244/244 [==============================] - ETA: 0s - loss: 0.5432 - accuracy: 0.7960 - sensitivity_at_specificity_1: 0.9959 - specificity_at_sensitivity_1: 0.9935 - recall_1: 0.7528 - precision_1: 0.8362
Epoch 14: val_accuracy did not improve from 0.71478
244/244 [==============================] - 176s 721ms/step - loss: 0.5432 - accuracy: 0

244/244 [==============================] - 176s 721ms/step - loss: 0.4425 - accuracy: 0.8357 - sensitivity_at_specificity_1: 0.9987 - specificity_at_sensitivity_1: 0.9968 - recall_1: 0.8056 - precision_1: 0.8645 - val_loss: 0.8987 - val_accuracy: 0.6959 - val_sensitivity_at_specificity_1: 0.9637 - val_specificity_at_sensitivity_1: 0.9723 - val_recall_1: 0.6499 - val_precision_1: 0.7345
Epoch 26/50
244/244 [==============================] - ETA: 0s - loss: 0.4365 - accuracy: 0.8357 - sensitivity_at_specificity_1: 0.9986 - specificity_at_sensitivity_1: 0.9972 - recall_1: 0.8063 - precision_1: 0.8652
Epoch 26: val_accuracy did not improve from 0.72904
244/244 [==============================] - 176s 721ms/step - loss: 0.4365 - accuracy: 0.8357 - sensitivity_at_specificity_1: 0.9986 - specificity_at_sensitivity_1: 0.9972 - recall_1: 0.8063 - precision_1: 0.8652 - val_loss: 0.8921 - val_accuracy: 0.7120 - val_sensitivity_at_specificity_1: 0.9665 - val_specificity_at_sensitivity_1: 0.9771 - v

Epoch 38/50
244/244 [==============================] - ETA: 0s - loss: 0.3331 - accuracy: 0.8738 - sensitivity_at_specificity_1: 0.9988 - specificity_at_sensitivity_1: 0.9990 - recall_1: 0.8546 - precision_1: 0.8938
Epoch 38: val_accuracy did not improve from 0.72904
244/244 [==============================] - 176s 722ms/step - loss: 0.3331 - accuracy: 0.8738 - sensitivity_at_specificity_1: 0.9988 - specificity_at_sensitivity_1: 0.9990 - recall_1: 0.8546 - precision_1: 0.8938 - val_loss: 1.1633 - val_accuracy: 0.6912 - val_sensitivity_at_specificity_1: 0.9452 - val_specificity_at_sensitivity_1: 0.9700 - val_recall_1: 0.6703 - val_precision_1: 0.7141
Epoch 39/50
244/244 [==============================] - ETA: 0s - loss: 0.3342 - accuracy: 0.8742 - sensitivity_at_specificity_1: 0.9987 - specificity_at_sensitivity_1: 0.9990 - recall_1: 0.8548 - precision_1: 0.8944
Epoch 39: val_accuracy did not improve from 0.72904
244/244 [==============================] - 176s 720ms/step - loss: 0.3342 -

In [18]:
#Acquiring Performance metrics

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity_1']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity_1']
Validation_Recall=history.history['val_recall_1']
Validation_Precision=history.history['val_precision_1']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.8998269438743591
Validation Accuracy:  0.7290355563163757
Validation Specificity:  0.9820533990859985
Validation Sensitivity:  0.9860023260116577
Validation Recall:  0.6881982684135437
Validation Precision:  0.7692908644676208
Validation Loss:  0.7420902252197266
